In [ ]:
!pip install datasets

In [10]:
from datasets import load_dataset

In [11]:
from datasets import load_dataset
emotions = load_dataset("emotion")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [12]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [13]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
def tokenize(batch):
  return tokenizer(
      batch['text'],
      padding=True,
      truncation=True
  )

In [15]:
emotions_encoded=emotions.map(
    tokenize,
    batched=True,
    batch_size=None
)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
emotions_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [17]:
emotions_encoded['train']['text'][0]

'i didnt feel humiliated'

In [18]:
emotions_encoded['train']['label'][0]

0

In [19]:
emotions_encoded['train']['input_ids'][0][:10]

[101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0]

In [20]:
emotions_encoded['train']['attention_mask'][0][:20]

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
from transformers import AutoModel
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)


In [31]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

In [32]:
def extract_hidden_states(batch):
    inputs = {k: v.to(device) for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    print("Last Hidden State Shape:", last_hidden_state.shape)  # Debugging line
    return {'hidden_state': last_hidden_state[:, 0].cpu().numpy()}


In [33]:
emotions_encoded.set_format('torch',columns=[
    'input_ids',
    'attention_mask',
    'label'
])

In [36]:
emotions_hidden=emotions_encoded.map(extract_hidden_states,batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])
Last Hidden State Shape: torch.Size([1000, 87, 768])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Last Hidden State Shape: torch.Size([1000, 69, 768])
Last Hidden State Shape: torch.Size([1000, 69, 768])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Last Hidden State Shape: torch.Size([1000, 66, 768])
Last Hidden State Shape: torch.Size([1000, 66, 768])


In [37]:
emotions_hidden

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2000
    })
})

In [39]:
emotions_hidden['train']['hidden_state'][0]

tensor([-1.1675e-01,  9.8571e-02, -1.2963e-01, -9.5018e-02, -2.8083e-01,
        -1.3222e-01,  2.2748e-01,  2.3100e-01,  1.6752e-01, -2.4663e-01,
         2.1755e-02, -1.0200e-01, -1.8098e-01,  2.2351e-01,  2.0275e-01,
         1.1693e-01, -6.9534e-04,  1.2025e-01, -1.4243e-04, -6.0743e-02,
        -1.4382e-01, -3.1201e-01, -1.0877e-01,  1.1821e-01, -7.3939e-02,
         7.2635e-02,  2.1319e-01, -2.9570e-01,  1.0432e-01, -1.4980e-01,
         1.1913e-01,  1.3559e-01, -2.0512e-01,  7.9983e-02, -1.2340e-01,
         2.4995e-02,  1.1948e-02,  1.0090e-02,  1.3740e-01, -1.3440e-02,
        -1.1266e-01, -7.7647e-02, -1.3007e-01, -7.1687e-02,  8.5297e-02,
        -5.4750e-02, -2.1881e+00, -5.0468e-02, -3.2211e-01, -1.6245e-01,
         3.1700e-01, -1.9575e-01,  1.3216e-01,  3.7076e-01, -5.4104e-02,
         3.5674e-01, -1.9445e-01,  3.1695e-01, -2.2347e-02,  9.1786e-02,
         2.9930e-01,  2.1970e-01, -1.4083e-01, -7.3253e-03, -7.7053e-02,
         2.2294e-01, -8.2599e-02,  3.5193e-01, -1.1

In [41]:
import numpy as np
X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])

In [43]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)
lr_clf.score(X_valid, y_valid)

0.633